# Greenlight presentation notes

## Current progress

tldr: HyperbolicKL has some issues, we switch to GaussianKL to overcome them (hopefully)

### HyperbolicKL notes
I'm trying to figure out what exactly is going wrong when certain flags are set:
1. Baseline experiment: (MNIST, wrong_grad, no scale_fix, HyperbolicKL, accel. on/off) \
   Works perfectly good. We get the results matching the paper (baseline)

2. Experiment 2: (MNIST, wrong_grad, scale_fix, HyperbolicKL, acce. on/off) \
   Works fairly well. We get results similar to baseline. But it requires (much) more iterations to converge nicely.
   This is due to the scaling of the inverse metric tensor term. Since it basically scales the gradients down consistently.

3. Experiment 3: (MNIST, correct_gad, no scale_fix, HyperbolicKL, accel. on/off) \
   Takes much longer to converge properly. The extra d^H_ij term makes gradients (near) 0 for a long time.
   Gradients get very large too at some point (embeddings overshoot the Disk)

4. Experiment 4: (MNIST, correct_grad, scale_fix, HyperbolicKL, accel. on/off) \
   Takes even longer to converge properly. Now we have both d^H_ij (initially) and scale_fix scaling the gradients down. \
   However now gradients stay within reasonable amounts and don't explode off to large values. Presumably due to the scale_fix \
   These settings reproduce the result where points are pushed towards the boundary (conceptual sensible one)

In points (3, 4) we get other issues. So the fact that gradients are very small means no progress is made for a long time, means the algorithm just stops.
\
If we use both fixes (gradient, scale_fix) then we get the "issue" that points are pushed towards the boundary. 

To avoid early stopping increase **n_iter_check** flag in **opt_config** and manually change **n_iter_without_progress** in **solver.py**

#### HyperbolicKL on Tree data
Does not work very well. Often points don't even move from the center or other weirdness happens. I haven't tested this specific case much yet though.

#### HyperbolicKL on Real data
Pushes points towards the boundaries much more strongly than GaussianKL
\
See experiment (73, 74)


### HyperbolicKL to GaussianKL Motivation

Gaussian has faster decreasing tails, resulting in forces being propegated out less, so embeddings are less likely to be pushed outwards towards the edge.

### GaussianKL notes

Some general notes:

- size_tol parameter and the Hyperbolic variance really affect the embedding. This requires some hand-tuning
- Early exaggeration 
- BH accel. numerical problems
- Large tree depth problems
- Crowding problem? Clusters being squished to points?
- GaussianKL gradient: (2/var term and learning rate)

#### GaussianKL on Tree dataset
On the tree dataset, GaussianKL seems to produce much better results than HyperbolicKL. Although there are some issues.

#### GaussianKL on Real datasets (MNIST, C_ELEGANS for now)
On the real datasets, GaussianKL does not push embeddings towards the boundaries as much as HyperbolicKL. 
\
See experiment (71, 72)

# Tree dataset

To properly test the notion that tree-like data can be nicely embedded in Hyperbolic Space, we turn to an artificially created tree-like dataset. 
\
I basically wrote some code that generates a Distance **D** and affinity **V** Matrix such that its entries reflect a tree-like ordering of the data. 

# Experiments

Some notable experiments and analysis:

## Laptop Experiments
#### Experiment 38 (also 68)
This experiment nicely showcases a tree embedding using GaussianKL. However, this experiment had an extra grad \*= 4 term in the cost function (leftover when I coped over the HyperbolicKL code) that caused the embedding to converge to this specific visualization.
\
\
If this term is removed, the clusters are more dense and converge to points. (experiment 66)
\
\
However, we can regain clusters by adjusting the learning rate (this basically mimics the grad \*=4 mistake).
It seems that adjusting the learning rate will give us "better" (subjective and context-dependent) embeddings.


#### Experiment 73. 74
Showcases HyperbolicKL on MNIST. Points are pushed along the boundary very strongly

#### Experiment 71, 72
Showcases GaussianKL on MNIST. Points are much less pushed toward the center and the embedding actually takes shape inside the visible part of the disk.

#### Experiment 83, 84
Attempt to reproduce 38, 68 without the grad \*= 4 term, but by adjusting the learning rate since it achieves basically the same result. This attempt was succesful. Therefore learning rate really affects the embeddings.

#### Experiment 85, 86
Experiment that showcases how adjusting learning rate also improves embeddings for larger trees. Retains tree-structure and still displays inter-cluster nodes nicely.
\
\
Exp 86: Experiment to see whether BH accel. improves performance. It indeed does improve performance and the embeddings still look nice.

#### Experiment 88, 89
C_ELEGANS dataset using GaussianKL, BH accel. on/off, and adjusted learning rates

#### Experiment 91
HyperbolicKL on a tree-like dataset. Embeddings are strongly pushed towards the boundary

#### Experiment 107, 108, 109
HyperbolicKL on MNIST data. To see how differing learning rate lead to possibly different convergings. 
\
Wrong Gradient, No scale_fix, exact embedding (like Hunter's work/original paper)
\
\
It seems that points are definitely pushed towards the edge.

#### Experiment 110, 111, 112
HyperbolicKL on MNIST data. Different learning rates. Same as (107, 108, 109) but not exact
\
Wrong Gradient, No scale_fix, not exact embedding
\
\
**Comparison between exact/non-exact**
\
109 vs 110: Smallest lr comparison. Embeddings look very similar. Clusters are very dense
\
107 vs 111: Medium lr comparison. Embeddings look very similar. Clusters are less dense
\
108 vs 112: Largest lr comparison. Embeddings look very similar. Clusters are the most dense.
\
\
It seems that lr. matters a lot in embedding quality. Too small or too large can produce too dense/crowded? clusters or embeddings. A sweet spot seems to be the medium lr (exag_factor * 100), or a lr around ~3.

#### Experiment 113, 114, 115
HyperbolicKL on C_ELEGANS dataset. Different learning rates.
\
Wrong Gradient, No scale_fix, not exact


## PC Experiments
#### PC Experiment 35, 36
HyperbolicKL on MNIST data. Different learning rates, (0.025 datasize)
\
Correct Gradient, Scale_fix, exact
\
\
Experiment to see what happens to embeddings when we differ the learning rates.
Embeddings get pushed towards the edge. 

#### PC Experiment 37, 38
HyperbolicKL on MNIST data. Different learning rates, (0.025 datasize)
\
Correct Gradient, Scale_fix, not exact
\
\
To see if embeddings look similar to 35, 36


# Future work

- Proper comparison of GaussianKL vs HyperbolicKL losses (on different data sets)
- Extra investigation into BH accel. for GaussianKL
- Possible "crowding" issues again with GaussianKL (although HyperbolicKL seems to have it too).
  Clusters basically become point-like (for tree datasets), which is strange since it is not the case in experiment 38
- Produce regular t-sne visualizations of Tree-like data
- Quantitative comparison of gradients 
- Comparing cost function between gradients
- Motivate why we should use the correct gradient; Compare "wrong" gradient with "right gradient objectives, what is being minimized in either cases? Do they both minimize the same objective?
- Writing; TU Delft writing center
- Structure all the goals, findings. Connect the contributions and progress, experiments that backup the contributions. Have a logical ordering from hunter's thesis, wrong gradient, to gaussian gradient and backup/connect the steps together. Identify my contributions in this chain and note them down for the thesis.
- Write down claims/goals explicitly, where is the story/thesis heading to. What are my claims?
- Add quantitative measurements for NE methods to substantiate my claims

- Time plan for this structure
- Aggregate important information into a file for teams
- Share overleaf link in teams

# Thesis structure

1. Introduction
    - **Introduce hyperbolic embeddings**
    Firsly we briefly introduce hyperbolic embeddings and its use cases. Explain connection between hierarchical relationships, tree/graph/network embeddings and hyperbolic space. Mention acceleration structure by Hunter.

    - **Explain my contributions, where the thesis is heading** \
    Secondly we talk about correcting the gradient:\
    We introduce a correction to the gradient(s) used in other work. Hint at methods/experiment section for proper justification but briefly state justifications here.
    \
    Then we talk about the Gaussian Gradient as a follow-up contribution for hyperbolic embeddings. Briefly explain why this contribution is significant.

    <!-- - **Why would people care** \
    Other versions used use a wrong gradient (possibly they didn't fully derive the gradient but just took euclidean t-sne gradient and substituted some terms). \
    Wrong gradient may lead to false insights as results do not optimize the correct cost function. Or rather, we don't know, can't fully explain what is being optimized. Link to experiments/methods where we empricially show (through analysis of cost function progression) that the 2 gradients act differently.
    -->

    - **Challenges** 
    TBD...


2. Related work (Straightforward)
    - data visualization (tsne)
    - Hyperbolic space & Visualizations
    - Hunters' work, other hyperbolic visualization
    - BH acceleration


3. Background (Straightforward)
    - Technical details about what is required to understand my thesis
    - Hyperbolic space, t-sne/neighbour embedding methods, gradient descent


4. Methods \
This section contains an overview of what I am contributing. Contains detailed reasons and motivations for what Im contributing (although empirical evidence is in the Experiments/Discussion sections).
    - **Correct Gradient** \
        Justify the change to the correct gradient as with the following:
        - Mathematical correctness argument, and bring into question what were even optimizing with the (wrong) gradient.
        - A correct mathematical derivation of the gradient (appendix?)
        - Empirical evidence of cost function convergence (to be elaborated on in Experiments section), to show that we are indeed not optimizing what we expect to be optimizing with the wrong gradient. (cost function)
    
    - **Tree-like data embedding** \
        Hyperbolic Space is supposed to embed trees (and graph/network-like) data well. (many references) \
        One contribution is an explicit experiment to test how trees are embedded using artificial tree-like data.
        - Explain this idea with a some detail. Why do we care about embedding a tree. 
        - Emphasize how we use tree-like data as further evidence for the correct gradient, and t-sne/gaussian gradient contributions.
        - Briefly go over how this data gets generated?
        - Show some visualizations 
        - Use quantifiable quality metrics (in experiments & discussion)

    - **Discuss limitations of t-sne distribution** \
        Justify this contribution step as follows:
        - Show how both the correct (and incorrect, aka original gradient used) gradient push points towards the boundary. \
        Originally, the incorrect gradient seems to not do this, but if you keep the algorithm running, it will also converge to a similarily pushed out embedding.
        - Explain why this is not desirable.
        - Explain where this comes from (heavy tails of t-distribution, repulsive forces being too strong, the nature and limitations of using Poincare Disk for visualization)
        - Visualization results & evaluation metric to further justify (in experiments & discussion)

    - **Proposal of Gaussian distribution usage** \
        Motivate gaussian distribution usage: 
        - Origina SNE paper used gaussian
        - t-distrib. was introduced for the crowding problem. Explain how in Hyperbolic space, this might be less of an issue (area/volume scales exponentially instead of linearly in 1D /quadratically in 2D)
        - Gaussian has flatter tails, so forces aren't felt a smuch. This can be demonstrated mathematically.
        - Visualization results & evaluation metric to further justify (in experiments & discussion)

    - **Integrating everything with Hunter's work** \
        - Link everything back to the acceleration structure build by Hunter.
        - Show that acceleration works equally as valid for Gaussian gradient.
        - Maybe more??
        

5. Experiments \
    Talk about experiments that motivate this thesis, connect them to my contributions/goals. Experiment starts with a question targetting a contribution/goal, answer question using the experiment, relate it to the next experiment

    - **E1: Correct Gradient justification experiments** \
    Firstly we compare hyperbolic t-sne correct vs. incorrect gradient:
        1. Cost function convergence graphs for several datasets incl. tree data
        2. Visual comparison of embeddings for several datasets 

    - **E2: Limitations of t-distrib. Gradient** 
        1. Show for (correct & wrong) gradient visualizations where embeddings are pushed out
        2. Quality metrics of t-distrib. gradient 
    
    - **E3: Improvents by Gaussian Gradient**
        1. Show (corrected) gaussian embeddings. Use same datasets as in t-distrib. section
        2. Quality metrics of gaussian gradient

    - **E4: Tree-data embeddings** \
    Maybe this can be used to further highlight a limitation of the t-distrib. gradient. So we put this section between **E2** and **E3**. 
        1. Show tree-data embedding for t-distrib. and gaussian gradient. \
           Produce results for varying "sized" trees. (differing in depth. children etc...)
        2. Quality metrics for this dataset
    
    - **E5: Acceleration structure for Gaussian gradient** \
    Investigate acceleration on Gaussian gradient embeddings? (TBD...)
        1. Proceed same as Hunter did in his thesis (TBD...)


6. Discussion
    - Discuss experiments and connect them together to shape a strong motivation for my goal
    - Focussed on content of thesis (methods/experiments)

    - **Use E1 to justify Correct Gradient** \
    Here we elaborate and fully justify the use of the correct gradient over the wrong one using **E1**
    We use the cost function graphs to highlight the problem with the wrong gradient (is not properly minimizing the objective in question), and show that the correct gradient does indeed minimize what we want. 

    - **E2 and t-distrib. limitations** \
    Here we use **E2** to convince the reader of the limitations of the t-distrib. gradient. We set the stage for the Gaussian gradient contribution. 

    - **E4: Tree-data justification** (maybe not as separate section) \
    Use tree-data embeddings to further justify Gaussian gradient over t-distrib. gradient. We can again use visuals, cost function comparisons, and quality retention metrics.
    We can also justify this with related works that talk about tree/graph/network data embedding in Hyperbolic space.

    - **E3: Gaussian gradient justification** \
    Now we show that the Gaussian gradient performs better in both visualization (and I assume also quality retention metrics).
    We compare visuals, cost function experiment results, quality retention metrics to justify the usage of the Gaussian gradient. 
    

7. Conclusion (TBD)
    - Link experiments, discussion back to the original questions, goals, theme of the thesis. 
    - Present recommendations, risks, guidelines based on experiments/discussion
    - Put things in context globally

# General notes?

1. Start with Hunters' work. Hyperbolic Embeddings in general. Try to obtain some results that bring doubt to claims in those original papers?
    - Wrong gradient vs Right gradient?
    - What is being optimized wrt. wrong gradient?
    - Is the actual cost function being minimized when we use the wrong gradient?
    - What is happening to the cost function when we use the wrong gradient?
    - How does this relate to the resulting embedding?
Talk about related works (Poincare map, co-sne, h-sne) that also use the wrong gradient. 


    - What goes on when we use the correct gradient?
    - Can we compare gradients? (think about this a bit more)
    - Why do we care about a correct gradient? (Hyperbolic space, capturing hierarchical structure)
    - Does the wrong gradient capture hierarchical structure?
    - Why do we care about the correct gradient?
    - TODO: Experiment with wrong gradient on various datasets

Also go into core motivations behind Hyperbolic embeddings. Are we capable of capturing hierarchy (visually) using Hunters' work? If not, why not? How do we know we're not? 
\
\
What is the end goal of such work? What kind of visualizations would we like to obtain? Do we have baseline experiments that can affirm this? (HyperbolicKL variants on tree-like data)

2. Lead insights, problems, "mistakes" from .1 to the focus of my thesis. 
    - What does Hunters' work etc.. fail to capture? What would we expect vs. what is being captured?
    - 

3. My proposals to "resolving" issues/mistakes/problems arising from .1 and .2
    - GaussianKL (and why?)